# Chapter 20 - Modular Monolith

- Meant to be a middle ground between monolith and microservice archtitectures
- Good for small to medium size projects
- Easier to transition from Monolith

## What is a Modular Monolith?

- Application organized into well-defined loosley coupled modules, responsible for a specific business capability
- Deployed as a single unit (unlike microservices)
- Treat each module as a microservice 
    * Seperate project / assembly

### Module aggregator 

- Ties the modules together
- Communication between modules
    * Event-driven arch is one option


### Design considerations

- Modules share a URL space
- Modules share the config infrastrucure
- Modules share a single dependency injection object graph (one container)
- Modules share inter-module communication infrastructure (event broker)


## Project - Modular Monolith

### Stack

- ASP.net Core
- Minimal API
- EF Core
- ExceptionMapper
- FluentValidation
- Mapperly
- Modules
    * REPR
    * `modules/{module namespace}/{module}`
- MassTransit (Event Broker)

### Data

- Can create a schmema (Module name) for each context to ensure they do not conflict with each other

```csharp
namespace REPR.Products.Data;
public class ProductContext : DbContext
{
    public ProductContext(DbContextOptions<ProductContext> options) : base(options) { }

    protected override void OnModelCreating(ModelBuilder modelBuilder)
    {
        base.OnModelCreating(modelBuilder);
        modelBuilder.HasDefaultSchema(Constants.ModuleName.ToLower());
    }

    public DbSet<Product> Products => Set<Product>();
}

namespace REPR.Baskets.Data;
public class BasketContext : DbContext
{
    public BasketContext(DbContextOptions<BasketContext> options) : base(options) { }
    public DbSet<BasketItem> Items => Set<BasketItem>();
    public DbSet<Product> Products => Set<Product>();

    protected override void OnModelCreating(ModelBuilder modelBuilder)
    {
        base.OnModelCreating(modelBuilder);
        modelBuilder.HasDefaultSchema(Constants.ModuleName.ToLower());
        modelBuilder
            .Entity<BasketItem>()
            .HasKey(x => new { x.CustomerId, x.ProductId })
            ;
    }
}
```

- Each schema has a Products table but they do not conflict

### Message broker

- [MassTransit](https://github.com/MassTransit/MassTransit) 
    * [Docs](https://masstransit.io/documentation/concepts)
    * Using InMemory provider
- Could have also used MediatR

### Operation summary

>  1. A client sends a POST request to the API (the aggregator application). The catalog module receives the request.
>  2. The catalog creates the product and adds it to the database.
>  3. The catalog then publishes the ProductCreated event using MassTransit.
>  4. In the basket module, the Consume method of the ProductEventsConsumers class gets called by MassTransit in reaction to the ProductCreated event.
>  5. The basket module adds the product’s identifier to the database; its materialized view.

### Publisher - Catalog module
C20\modules\catalog\Products\Features\CreateProduct\CreateProductHandler.cs

- Passes `CancellationToken.None` so the operation cannot be cancelled by consuming code

```csharp
namespace Products.Features;

public class CreateProductHandler
{
    private readonly ProductContext _db;
    private readonly CreateProductMapper _mapper;
    private readonly IBus _bus;

    public CreateProductHandler(ProductContext db, CreateProductMapper mapper, IBus bus)
    {
        _db = db ?? throw new ArgumentNullException(nameof(db));
        _mapper = mapper ?? throw new ArgumentNullException(nameof(mapper));
        _bus = bus ?? throw new ArgumentNullException(nameof(bus));
    }

    public async Task<CreateProductResponse> HandleAsync(CreateProductCommand command, CancellationToken cancellationToken)
    {
        var product = _mapper.Map(command);
        var entry = _db.Products.Add(product);
        await _db.SaveChangesAsync(cancellationToken);

        var productCreated = _mapper.MapToIntegrationEvent(entry.Entity);
        await _bus.Publish(productCreated, CancellationToken.None);

        var response = _mapper.MapToResponse(entry.Entity);
        return response;
    }
}
```

### Consumer - Cart module

C20\modules\cart\Baskets\Features\ProductEventsConsumers.cs

```csharp
namespace Baskets.Features;

public class ProductEventsConsumers : IConsumer<ProductCreated>, IConsumer<ProductDeleted>
{
    private readonly BasketContext _db;
    private readonly ILogger _logger;
    public ProductEventsConsumers(BasketContext db, ILogger<ProductEventsConsumers> logger)
    {
        _db = db ?? throw new ArgumentNullException(nameof(db));
        _logger = logger ?? throw new ArgumentNullException(nameof(logger));
    }

    public async Task Consume(ConsumeContext<ProductCreated> context)
    {
        _logger.LogTrace("Consuming {eventName} {eventId}", nameof(ProductCreated), context.MessageId);
        var product = await _db.Products.FirstOrDefaultAsync(
            x => x.Id == context.Message.Id,
            cancellationToken: context.CancellationToken
        );
        if (product is not null)
        {
            _logger.LogWarning("The product {productId} already exist.", context.Message.Id);
            return;
        }
        _db.Products.Add(new(context.Message.Id));
        await _db.SaveChangesAsync();
        _logger.LogInformation("The product {productId} was created successfully.", context.Message.Id);
        _logger.LogTrace("Consumed {eventName} {eventId}", nameof(ProductCreated), context.MessageId);
    }

    ...
}
```

### Trasnsitioning to microservices

- If a module is moved to a microservice the gateway could easily reroute requests without the clients needing to change

- Keep as a monolith unless there is a good reason not to

### Challenges

- Modules becomes too complex
    * Make sure the scope of the module is limited to one business concern
    * Dont be afraid of refactoring
- Poorly defined module boundaries
    * Good planning and domain analsysis
    * SRP
    * Refactor
- Scaling 
    * Monoliths do not have the flexability to scale like microservices
    * Migrate to a microservices
- Eventual consistency
    * Use a resilient broker in production (Like Kafka)
- Transition to microserivces
    * Use event-driven architecture can make the transition less painful